In [ ]:
# default_exp core
# default_cls_lvl 3

In [ ]:
#hide
%load_ext line_profiler
%matplotlib widget

# Corefunctions
> Corefunctionality for data preparation of sequential data for pytorch, fastai models

# Application Structure
The data will be extracted and prepared via transforms. Those are grouped in:
- Type Transforms: Those extraxt the needed components from the source items, like input sequences or target scalar values. The work on single tensors.
- Item Transforms: Those Transforms may work on tuple level and therefore may process relationships between input and output.
- Batch Transform: Those transforms work on batch level. They receive batched tensors and may apply lazy transforms like normalization very effeciently.

An application example may look like the following:
- sourceitems: 
    - path extraction with hdf5 file endings
    - create pandas dataframe with information for type transforms, like slices
    - filter items in pandas dataframe
- type transforms: 
    - extract hdf5 input and output sequence
    - create windows
- item transforms: 
    - filter sequence by value
    - shift output sequence by 1 element
- batch transforms: 
    - noise injection
    - normalization
    

In [ ]:
#export
from fastai.data.all import *
from fastai.vision.augment import RandTransform
import h5py

In [ ]:
#export
def obj_in_lst(lst,cls):
    '''retrieve first object of type cls from a list'''
    return next(o for o in lst if type(o) is cls)

In [ ]:
#export
def count_parameters(model):
    '''retrieve number of trainable parameters of a model'''
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## 1. Extract Source Items
The file paths may be extracted with `get_files` of fastai2. `get_hdf_files` removes the need of writing the hdf5 file extension.

Then a pandas dataframe may be created in case further information for the source items need to be stored like slices for the windowing function.

### 1.1 Extract File Paths

In [ ]:
f_path = 'test_data/'
hdf_files = get_files(f_path,extensions='.hdf5',recurse=True)
len(hdf_files),hdf_files[0]

(3, Path('test_data/train/Sim_RealisticCycle1.hdf5'))

In [ ]:
#export
hdf_extensions = ['.hdf5']
def get_hdf_files(path,recurse=True, folders=None):
    "Get hdf5 files in `path` recursively, only in `folders`, if specified."
    return get_files(path, extensions=hdf_extensions, recurse=recurse, folders=folders)

In [ ]:
hdf_files = get_hdf_files(f_path)
len(hdf_files),hdf_files[0]

(3, Path('test_data/train/Sim_RealisticCycle1.hdf5'))

### 1.2 Create Source Dictionaries
In order to extract mulitple realizations of one file with different modifications, we create a list of properties. Pandas Dataframes are to slow for iteration but very fast and convenient for creations. So after creation of the pandas Dataframe we convert it to a list of dictionaries.

In [ ]:
#export
def apply_df_tfms(src,pd_tfms = None):
    '''Create Pandas Dataframe out of a list of items, with a list of df transforms applied'''
    if type(src) is pd.DataFrame:
        df = src
    else:
        df = pd.DataFrame(data=src.items,columns=['path'],dtype=str)
    if pd_tfms is not None:
        for t in pd_tfms:
            df = t(df)
    return df

In [ ]:
df = apply_df_tfms(hdf_files)
df.head()

,path
0,test_data/train/Sim_RealisticCycle1.hdf5
1,test_data/train/Sim_RealisticCycle2.hdf5
2,test_data/valid/Sim_RealisticCycle3.hdf5


In [ ]:
test_eq(apply_df_tfms(hdf_files),apply_df_tfms(apply_df_tfms(hdf_files)))

In [ ]:
#export
def CreateDict(pd_tfms = None):
    '''Create List of Dictionarys out of a list of items, with a list of df transforms applied'''
    def _inner(src):
        df = apply_df_tfms(src,pd_tfms)
#         df_dict_list = df.to_dict(orient='records') native to_dict is slower than self written approach
        df_values = df.values
        df_dict = {name:list(df_values[:,i]) for (i,name) in enumerate(df.columns)}
        df_dict_list = [{name: df_dict[name][i] for name in df_dict} for i in range(len(df))]
        return df_dict_list
    return _inner

In [ ]:
l_dict =CreateDict()(hdf_files)
l_dict

[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5'},
 {'path': 'test_data/train/Sim_RealisticCycle2.hdf5'},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5'}]

In [ ]:
#export
def ValidClmContains(lst_valid):
    '''add validation column using a list of strings that are part of the validation frames'''
    def _inner(df):
        re_valid = '|'.join([re.escape(f) for f in lst_valid])
        df['valid'] = df.path.str.contains(re_valid)
        return df

    return _inner

In [ ]:
%%time
lst_valid = ['valid']
CreateDict([ValidClmContains(lst_valid)])(hdf_files)

CPU times: user 2.87 ms, sys: 0 ns, total: 2.87 ms
Wall time: 2.85 ms


[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5', 'valid': False},
 {'path': 'test_data/train/Sim_RealisticCycle2.hdf5', 'valid': False},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5', 'valid': True}]

In [ ]:
#export
def ValidClmIs(lst_valid):
    '''adds validation column using a list of validation filenames'''
    def _inner(df):
        df['valid'] = df.path.isin([str(f) for f in lst_valid])
        return df

    return _inner

In [ ]:
%%time
lst_valid = ['test_data/train/Sim_RealisticCycle2.hdf5','test_data/valid/Sim_RealisticCycle3.hdf5']
CreateDict([ValidClmIs(lst_valid)])(hdf_files)

CPU times: user 2.43 ms, sys: 0 ns, total: 2.43 ms
Wall time: 2.42 ms


[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5', 'valid': False},
 {'path': 'test_data/train/Sim_RealisticCycle2.hdf5', 'valid': True},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5', 'valid': True}]

In [ ]:
#export
def FilterClm(clm_name,func = lambda x:x):
    '''adds validation column using a list of validation filenames'''
    def _inner(df):
        return df[func(df[clm_name])]

    return _inner

In [ ]:
CreateDict([ValidClmIs(lst_valid),FilterClm('valid')])(hdf_files)

[{'path': 'test_data/train/Sim_RealisticCycle2.hdf5', 'valid': True},
 {'path': 'test_data/valid/Sim_RealisticCycle3.hdf5', 'valid': True}]

In [ ]:
#export
def get_hdf_seq_len(f_path,clm):
    '''extract the sequence length of the dataset with the 'clm' name and 'f_path' path  '''
    with h5py.File(f_path,'r') as f:
        f_len = max(f[clm].shape)
    return f_len 

In [ ]:
#export
def df_get_hdf_seq_len(df,clm):
    '''extracts the sequence length of every file in advance to prepare repeated window extractions with 'DfHDFCreateWindows' '''
#     df['seq_len'] = ([get_hdf_seq_len(row.path,clm) for (idx, row) in df.iterrows()])
    df['seq_len'] = df.path.apply(lambda x: get_hdf_seq_len(x,clm))
    return df

In [ ]:
#export
def DfHDFGetSeqLen(clm):
    def _inner(df):
        return df_get_hdf_seq_len(df,clm)
    return _inner

In [ ]:
df_get_hdf_seq_len(df,'current')

,path,seq_len
0,test_data/train/Sim_RealisticCycle1.hdf5,265607
1,test_data/train/Sim_RealisticCycle2.hdf5,265598
2,test_data/valid/Sim_RealisticCycle3.hdf5,265593


In [ ]:
DfHDFGetSeqLen('current')(df)

,path,seq_len
0,test_data/train/Sim_RealisticCycle1.hdf5,265607
1,test_data/train/Sim_RealisticCycle2.hdf5,265598
2,test_data/valid/Sim_RealisticCycle3.hdf5,265593


In [ ]:
#export
import numbers
def DfResamplingFactor(src_fs,lst_targ_fs):
    if not isinstance(src_fs, numbers.Number) and not type(src_fs) is str: 
        raise ValueError('src_fs has to be a column name or a fixed number')
    
    def _inner(df):
        np_targ_fs = array(lst_targ_fs)
        pd.options.mode.chained_assignment = None #every row is a reference so we need to suppress the warning messages while copying

        #repeat entries for every target fs
        res_df = df.iloc[np.repeat(np.arange(len(df)),len(np_targ_fs))] 
        targ_fs = np.tile(np_targ_fs,len(df))
        res_df['targ_fs'] = targ_fs
        
        if isinstance(src_fs, numbers.Number):
            #src_fs is a fixed number
            res_df['resampling_factor'] = targ_fs/src_fs
        else:
            #src_fs is a column name of the df
            res_df['resampling_factor'] = targ_fs/res_df[src_fs]

        pd.options.mode.chained_assignment = 'warn'
        
        return res_df
    return _inner

In [ ]:
targ_fs = [50,100,300]
test_eq(len(DfResamplingFactor(100,targ_fs)(df)),9)   
df['src_fs'] = 200.
test_eq(len(DfResamplingFactor('src_fs',targ_fs)(df)),9)

In [ ]:
#export
def DfHDFCreateWindows(win_sz,stp_sz, clm, fixed_start = False, fixed_end = False):
    '''create windows of sequences, splits sequence into multiple items'''
    def _inner(df):
        if fixed_start and fixed_end: raise Exception
        
        if 'seq_len' in df:
            np_f_len = df.seq_len.values
        else:
            np_f_len = np.array([get_hdf_seq_len(row.path,clm) for (idx, row) in df.iterrows()])
            
        if 'resampling_factor' in df: np_f_len =(np_f_len*df.resampling_factor.values).astype(int)
            
        n_win = ((np_f_len-win_sz)//stp_sz)+1
        n_win = np.clip(n_win,a_min=0,a_max=None) #remove negative values at instances where the winsize is smaller than the seq_len
        lst_idx = np.arange(len(np_f_len))
        
        pd.options.mode.chained_assignment = None #every row is a reference so we need to suppress the warning messages while copying
        
        res_df = df.iloc[np.repeat(lst_idx,n_win)]
#         res_df = df.loc[np.repeat(lst_idx,n_win)] #the loc variant as a little bit slower because it creates copies and returns wrong values with redundant indexes, but is more robust

        step_idx = np.concatenate([np.arange(x) for x in n_win])
    
        
        res_df['l_slc'] = step_idx*stp_sz if not fixed_start else None
        res_df['r_slc'] = step_idx*stp_sz + win_sz if not fixed_end else None
            
        pd.options.mode.chained_assignment = 'warn'
            
        return res_df
    
    return _inner

In [ ]:
%%time
create_win = DfHDFCreateWindows(win_sz=100,stp_sz=100,clm='current')
win_df = create_win(df)
win_df

CPU times: user 4.02 ms, sys: 23 µs, total: 4.05 ms
Wall time: 3.67 ms


,path,seq_len,src_fs,l_slc,r_slc
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,0,100
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,100,200
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,200,300
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,300,400
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,400,500
...,...,...,...,...,...
2,test_data/valid/Sim_RealisticCycle3.hdf5,265593,200.0,265000,265100
2,test_data/valid/Sim_RealisticCycle3.hdf5,265593,200.0,265100,265200
2,test_data/valid/Sim_RealisticCycle3.hdf5,265593,200.0,265200,265300
2,test_data/valid/Sim_RealisticCycle3.hdf5,265593,200.0,265300,265400


In [ ]:
win_df = DfHDFCreateWindows(win_sz=265594,stp_sz=100,clm='current')(df)
test_eq(len(win_df),2)
win_df

,path,seq_len,src_fs,l_slc,r_slc
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,0,265594
1,test_data/train/Sim_RealisticCycle2.hdf5,265598,200.0,0,265594


In [ ]:
test_eq(create_win(df_get_hdf_seq_len(df,'current')) , create_win(df))

In [ ]:
res_win_df = create_win(DfResamplingFactor(200,[0.1])(df))
res_win_df

,path,seq_len,src_fs,targ_fs,resampling_factor,l_slc,r_slc
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,0.1,0.0005,0,100
1,test_data/train/Sim_RealisticCycle2.hdf5,265598,200.0,0.1,0.0005,0,100
2,test_data/valid/Sim_RealisticCycle3.hdf5,265593,200.0,0.1,0.0005,0,100


In [ ]:
test_eq(len(res_win_df),3)

In [ ]:
query_expr = 'l_slc <= 200'
filt_df = win_df.query(query_expr)
filt_df

,path,seq_len,src_fs,l_slc,r_slc
0,test_data/train/Sim_RealisticCycle1.hdf5,265607,200.0,0,265594
1,test_data/train/Sim_RealisticCycle2.hdf5,265598,200.0,0,265594


In [ ]:
#export
def DfFilterQuery(query):
    def _inner(df):
        return df.query(query)
    return _inner

In [ ]:
test_eq(DfFilterQuery(query_expr)(win_df),filt_df)

In [ ]:
%%time
tfm_src = CreateDict([ValidClmContains(['valid']),DfHDFCreateWindows(win_sz=100+1,stp_sz=10,clm='current')])
src_dicts = tfm_src(hdf_files)
src_dicts[:5]

CPU times: user 75.8 ms, sys: 15.1 ms, total: 90.9 ms
Wall time: 89.9 ms


[{'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 0,
  'r_slc': 101},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 10,
  'r_slc': 111},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 20,
  'r_slc': 121},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 30,
  'r_slc': 131},
 {'path': 'test_data/train/Sim_RealisticCycle1.hdf5',
  'valid': False,
  'l_slc': 40,
  'r_slc': 141}]

In [ ]:
#export
def DfDropClmExcept(clms = ['path','l_slc','r_slc','p_sample','resampling_factor']):
    '''drop unused dataframe columns as a last optional step to accelerate dictionary conversion'''
    def _inner(df):
        return df[[c for c in clms if c in df]]
    return _inner

## 2. Convert Paths to Sequence Objects
Der Pfad wird unter Angabe der Spaltennamen in Sequenzen und Skalare Werte umgewandelt, um so am Ende ein 3-Tupel zu erhalten aus:
- (Sequence, Scalar, Sequence) <-> (input,input,output)

### 2.1 Extract sequential data from hdf5-files
Two different functions, based on pandas df and on lists

#### 2.1.1 Shift time Series
Sometimes we need to shift columns of a sequence by a specific value. Then we cant simply slice the array but have to handle each column individually. First a performance test has to be made.

In [ ]:
#export
def calc_shift_offsets(clm_shift):
    clm_shift = array(clm_shift)
    l_offs = -min(clm_shift.min(),0)
    r_offs = -max(clm_shift.max(),0)
    l_shift = clm_shift+l_offs
    r_shift = clm_shift+r_offs
    dim_red = l_offs-r_offs
    return l_shift,r_shift,dim_red

In [ ]:
shft = [0,0,-1,1]
calc_shift_offsets(shft)

(array([1, 1, 0, 2]), array([-1, -1, -2,  0]), 2)

both shifting methods have their own performance character. vstack needs double the time on short sequences, while the creation of a seperate array with copy becomes worse starting at around 5000 elements

In [ ]:
# ta = array([[1,2,3]*2]*10000)

In [ ]:
# %%timeit
# y = np.vstack([ta[i:-ta.shape[1]+i,i] for i in range(ta.shape[1])]).T   

In [ ]:
# %%timeit
# x = np.zeros((ta.shape[0]-ta.shape[1],ta.shape[1]))
# for i in range(ta.shape[1]):
#     x[:,i] = ta[i:-ta.shape[1]+i,i]

#### 2.1.2 HDF2Sequence
HDF5 performance is massively affected by the dtype of the signals. f4 (32 bit floating point) Numbers are faster to load and lead to smaller files then f8 numbers.

In [ ]:
#export
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0,axis=0),axis=0) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
#export
def downsample_mean(x,N):
    shp = x.shape
    trunc = -(x.shape[0] % N)
    trunc = trunc if trunc != 0 else None
    return x[:trunc,:].reshape((-1,N,x.shape[-1])).mean(axis=1)

In [ ]:
#export
from scipy.signal import butter, lfilter, lfilter_zi
from scipy import signal
def resample_interp(x,resampling_factor,sequence_first=True, lowpass_cut=0.7, upsample_cubic_cut = None):
    '''signal resampling using linear or cubic interpolation
    
    x: signal to resample with shape: features x resampling_dimension or resampling_dimension x  features if sequence_first=True
    resampling_factor: Factor > 0 that scales the signal
    lowpass_cut: Upper boundary for resampling_factor that activates the lowpassfilter, low values exchange accuracy for performance, default is 0.7
    upsample_cubic_cut: Lower boundary for resampling_factor that activates cubic interpolation at high upsampling values. 
                        Improves signal dynamics in exchange of performance. None deactivates cubic interpolation
    '''
    
    if sequence_first:
        x = x.T
    
    fs_n = resampling_factor
    #if downsampling rate is too high, lowpass filter before interpolation
    if fs_n < lowpass_cut:
        b,a = butter(2, fs_n)
        zi = lfilter_zi(b,a)*x[:,:1] #initialize filter with steady state at first time step value
        x,_ = lfilter(b,a,x,axis=-1,zi=zi)

#         sos = butter(2, fs_n*1.2,output='sos')
# #         sos = signal.cheby2(2,20, fs_n,output='sos')
# #         import pdb;pdb.set_trace()
#         zi = np.swapaxes(signal.sosfilt_zi(sos)[...,None]*x[:,0],1,2)
#         x,_ = signal.sosfilt(sos, x,axis=-1,zi=zi)
        
    x_int = tensor(x)[None,...]
    targ_size = int(x.shape[-1]*fs_n)
    
#     if upsampling rate is too high, switch from linear to cubic interpolation
    if upsample_cubic_cut is None or fs_n <= upsample_cubic_cut:
        x = array(nn.functional.interpolate(x_int, size=targ_size, mode='linear',align_corners=False)[0])
    else:
        x = array(nn.functional.interpolate(x_int[...,None], size=[targ_size,1], mode='bicubic',align_corners=False)[0,...,0])
#     x = array(x_int)[0]
    
    if sequence_first:
        x = x.T
    
    return x

In [ ]:
x = np.random.normal(size=(100000,9))
test_eq(resample_interp(x,0.3).shape[0],30000)

In [ ]:
# def plotFFT(buffer,sample_rate,label,fig=plt):
#     xf,yf = calcFFT(buffer,sample_rate)
#     fig.plot(xf, yf,label=label)
    
# def calcFFT(buffer,sample_rate):
#     N = buffer.shape[0]
#     T = 1.0 / sample_rate
#     x = np.linspace(0.0, N*T, N)
#     y = buffer
#     yf = scipy.fft.fft(y)
#     xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
#     return xf,2.0/N * np.abs(yf[:N//2])

# src_fs= 200
# n_fs = 0.9
# plt.figure()
# plotFFT(x[:,0],src_fs,'raw')
# plotFFT(resample_interp(x,n_fs)[:,0],n_fs*src_fs,'filtered')
# plt.legend()

In [ ]:
#export
def hdf_extract_sequence(hdf_path,clms,dataset = None, l_slc = None, r_slc= None, resampling_factor=None, fs_idx =None,dt_idx =False):
    '''
    extracts a sequence with the shape [seq_len x num_features]
    
    hdf_path: file path of hdf file, may be a string or path type
    clms: list of dataset names of sequences in hdf file
    dataset: dataset root for clms. Useful for multiples sequences stored in one file.
    l_slc: left boundary for extraction of a window of the whole sequence
    r_slc: right boundary for extraction of a window of the whole sequence
    resampling_factor: scaling factor for the sequence length, uses 'resample_interp' for resampling
    fs_idx: clms list idx of fs entry in sequence. Will be scaled by resampling_factor after resampling
    dt_idx: clms list idx of dt entry in sequence. Will be scaled by resampling_factor after resampling
    '''

    if resampling_factor is not None:
        seq_len = r_slc-l_slc if l_slc is not None and r_slc is not None else None #calculate seq_len for later slicing, necesary because of rounding errors in resampling
        if l_slc is not None: l_slc= math.floor(l_slc/resampling_factor)
        if r_slc is not None: r_slc= math.ceil(r_slc/resampling_factor)

    with h5py.File(hdf_path,'r') as f:
        ds = f if dataset is None else f[dataset]
        l_array = [(ds[n][l_slc:r_slc]) for n in clms]
        seq = np.stack(l_array,axis=-1)

    if resampling_factor is not None:
        res_seq = resample_interp(seq,resampling_factor)
        if fs_idx is not None: res_seq[:,fs_idx] = seq[0,fs_idx] * resampling_factor
        if dt_idx is not None: res_seq[:,dt_idx] = seq[0,dt_idx] / resampling_factor
        seq = res_seq
        
        if seq_len is not None: seq = seq[:seq_len] #cut the part of the sequence that is too long because of resampling rounding errors
        
    return seq

In [ ]:
#export
class Memoize:
    def __init__(self, fn):
        self.fn = fn
        self.memo = {}

    def __call__(self, *args):
        if args not in self.memo:
            self.memo[args] = self.fn(*args)
        return self.memo[args]

In [ ]:
#export
class HDF2Sequence(Transform):
    
    def __init__(self, clm_names,clm_shift=None,truncate_sz=None,to_cls=noop,cached=False, fs_idx =None,dt_idx =False):
        if not clm_shift is None:
            assert len(clm_shift)==len(clm_names) and all(isinstance(n, int) for n in clm_shift)
            self.l_shift,self.r_shift,_ = calc_shift_offsets(clm_shift)
        
        self._exseq = Memoize(self._hdf_extract_sequence) if cached else self._hdf_extract_sequence
        
        store_attr('clm_names,clm_shift,truncate_sz,to_cls,cached,fs_idx,dt_idx')
        
    def _hdf_extract_sequence(self,hdf_path,dataset = None, l_slc = None, r_slc= None, resampling_factor=None, fs_idx =None,dt_idx =False):
        '''
        extracts a sequence with the shape [seq_len x num_features]

        hdf_path: file path of hdf file, may be a string or path type
        clms: list of dataset names of sequences in hdf file
        dataset: dataset root for clms. Useful for multiples sequences stored in one file.
        l_slc: left boundary for extraction of a window of the whole sequence
        r_slc: right boundary for extraction of a window of the whole sequence
        resampling_factor: scaling factor for the sequence length, uses 'resample_interp' for resampling
        fs_idx: clms list idx of fs entry in sequence. Will be scaled by resampling_factor after resampling
        dt_idx: clms list idx of dt entry in sequence. Will be scaled by resampling_factor after resampling
        '''

        if resampling_factor is not None:
            seq_len = r_slc-l_slc if l_slc is not None and r_slc is not None else None #calculate seq_len for later slicing, necesary because of rounding errors in resampling
            if l_slc is not None: l_slc= math.floor(l_slc/resampling_factor)
            if r_slc is not None: r_slc= math.ceil(r_slc/resampling_factor)

        with h5py.File(hdf_path,'r') as f:
            ds = f if dataset is None else f[dataset]
            l_array = [(ds[n][l_slc:r_slc]) for n in self.clm_names]
            seq = np.stack(l_array,axis=-1)

        if resampling_factor is not None:
            res_seq = resample_interp(seq,resampling_factor)
            if fs_idx is not None: res_seq[:,fs_idx] = seq[0,fs_idx] * resampling_factor
            if dt_idx is not None: res_seq[:,dt_idx] = seq[0,dt_idx] / resampling_factor
            seq = res_seq

            if seq_len is not None: seq = seq[:seq_len] #cut the part of the sequence that is too long because of resampling rounding errors

        return seq
    
    def _extract_dict_sequence(self,item):
        if isinstance(item,dict):
            path = item['path']
            dataset = item['dataset'] if 'dataset' in item else None
            l_slc = item['l_slc'] if 'l_slc' in item else None
            r_slc = item['r_slc'] if 'r_slc' in item else None
            resampling_factor = item['resampling_factor'] if 'resampling_factor' in item else None

            if self.cached:
                seq = self._exseq(path,dataset,None,None,resampling_factor,self.fs_idx,self.dt_idx)[l_slc:r_slc]
            else:
                seq = self._exseq(path,dataset,l_slc,r_slc,resampling_factor,self.fs_idx,self.dt_idx)
        else:
            seq = self._exseq(str(item),None,None,None,None)

        #shift clms of result by given value 
        if not self.clm_shift is None:
            l_seq = seq.shape[0]
            seq = np.stack([seq[self.l_shift[i]:l_seq+self.r_shift[i],i] for i in range(seq.shape[1])],axis=-1)
            
        if not self.truncate_sz is None:
            seq = seq[truncate_sz:]
        
        #it is important to slice first and then do the class conversion
#         return self.to_cls(seq.astype('f8'))#workaround for random bug, that mitigates convergence if the numpy array is an f4 array. Seems to make no sense because the result does not change. 
        return self.to_cls(seq)

    def encodes(self, item)->None: 
        return self._extract_dict_sequence(item)

In [ ]:
# %%timeit
hdf2seq = HDF2Sequence(['current','voltage'],cached=False)
hdf2seq(hdf_files[0])

array([[ 0.       ,  4.1873503],
       [-0.1      ,  4.18935  ],
       [-0.1      ,  4.1896954],
       ...,
       [ 8.8388   ,  3.3932123],
       [ 8.846    ,  3.3928714],
       [ 8.8531   ,  3.3925302]], dtype=float32)

In [ ]:
hdf2seq = HDF2Sequence(['current','voltage'],clm_shift=[1,1])
hdf2seq(hdf_files[0])

array([[-0.1      ,  4.18935  ],
       [-0.1      ,  4.1896954],
       [-0.1      ,  4.190037 ],
       ...,
       [ 8.8388   ,  3.3932123],
       [ 8.846    ,  3.3928714],
       [ 8.8531   ,  3.3925302]], dtype=float32)

In [ ]:
hdf2seq = HDF2Sequence(['current','voltage'],cached=False)
hdf2seq(hdf_files[0])

array([[ 0.       ,  4.1873503],
       [-0.1      ,  4.18935  ],
       [-0.1      ,  4.1896954],
       ...,
       [ 8.8388   ,  3.3932123],
       [ 8.846    ,  3.3928714],
       [ 8.8531   ,  3.3925302]], dtype=float32)

In [ ]:
%%timeit
hdf2seq(hdf_files[0])

21.4 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
hdf2seq = HDF2Sequence(['current','voltage'],cached=True)

In [ ]:
# %%timeit
hdf2seq(hdf_files[0])

array([[ 0.       ,  4.1873503],
       [-0.1      ,  4.18935  ],
       [-0.1      ,  4.1896954],
       ...,
       [ 8.8388   ,  3.3932123],
       [ 8.846    ,  3.3928714],
       [ 8.8531   ,  3.3925302]], dtype=float32)

Die Funktion lässt sich mittels Pipeline auf eine Liste von Quellobjekten (hier Pfade) anwenden 

In [ ]:
hdf2seq = HDF2Sequence(['current'])
hdf2seq(hdf_files[0]).shape

(265607, 1)

In [ ]:
pipe = Pipeline(HDF2Sequence(['current','voltage']))

In [ ]:
# res_pipe = pipe(hdf_files)
# len(res_pipe), res_pipe[0][0]

In [ ]:
#export
def hdf2scalars(hdf_path,c_names):
    with h5py.File(hdf_path,'r') as f:
#         import pdb; pdb.set_trace()
#         l_array = [f[n][:][:,None] for n in c_names]
#         seq = np.concatenate(l_array,axis=1)
        return None

#### Performance Test
Caching stores the arrays for future use at every function call. Very usefull, especially for windows. Should allways be turned. Only explicitly turn it off when there is not enough memory for your data.

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage'],cached=False)],
        [HDF2Sequence(['voltage'],cached=False)]]
dsrc = Datasets(src_dicts[:1000],tfms=tfms)

In [ ]:
len(dsrc)

1000

In [ ]:
# %%time
# for x in dsrc:
#     x

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage'],cached=True,clm_shift=[1,2])],
        [HDF2Sequence(['voltage'],cached=True)]]
dsrc = Datasets(src_dicts[:1000],tfms=tfms)

In [ ]:
# %%timeit
for x in dsrc:
    x

Caching is way faster because every file gets loaded multiple times

### 2.1 Datatypes for Sequences and Scalars

In [ ]:
#export
class TensorSequences(TensorBase):#TensorBase
#     def __init__(self,x,c_names=None, **kwargs):
#         super().__init__()
#         self.c_names = c_names
    
    def show(self, ctx=None, **kwargs):
#         import pdb; pdb.set_trace()
        ax = ctx
        if ax is None: _,ax = plt.subplots()
        ax.plot(self)
#         if title is not None: ax.set_title(title)
        return ax

    @classmethod
    @delegates(HDF2Sequence, keep=True)
    def from_hdf(cls,clm_names,**kwargs):
        return HDF2Sequence(clm_names,**kwargs)
    
class TensorSequencesInput(TensorSequences): pass
class TensorSequencesOutput(TensorSequences): pass

In [ ]:
f = TensorSequencesInput.from_hdf(['current'])
type(f(hdf_files[0]))

numpy.ndarray

In [ ]:
# TensorSequences(np.ones((30,2))).show()

In [ ]:
#export
@Transform
def toTensorSequencesInput(o): return TensorSequencesInput(o)
@Transform
def toTensorSequencesOutput(o): return TensorSequencesOutput(o)

In [ ]:
#export
class TensorScalars(Tensor): pass
class TensorScalarsInput(TensorScalars): pass
class TensorScalarsOutput(TensorScalars): pass

## 3. Transformations


### 3.1 Sequence Slicing Transformation

In [ ]:
#export 
class SeqSlice(Transform):
    '''Take a slice from an array-like object. Useful for e.g. shifting input and output'''
    def __init__(self, l_slc=None,r_slc=None):
        self.l_slc,self.r_slc = l_slc,r_slc
        
    def encodes(self, o): return o[self.l_slc:self.r_slc]

In [ ]:
l_shift = SeqSlice(r_slc=-1)
arr = np.ones((5))
test_eq(l_shift(arr),arr[:-1])

### 3.2 Sequence Noise Injection Transformation

In [ ]:
#export
class SeqNoiseInjection(RandTransform):
    split_idx=0
    '''Adds normal distributed noise to the tensor sequence with seperate mean and std for every signal'''
    def __init__(self, std=1e-1,mean=0.,p=1.0):
        super().__init__(p=p)
        self.std = tensor(std).type(torch.float)
        self.mean = tensor(mean).type(torch.float)
        
    def encodes(self, o:TensorSequencesInput): 
        if o.device != self.mean.device:
            self.std = self.std.to(o.device)
            self.mean = self.mean.to(o.device)
        #expand creates a view on a tensor and is therefore very fast compared to copy
        return o+torch.normal(mean=self.mean.expand_as(o), 
                              std=self.std.expand_as(o))

In [ ]:
x = TensorSequencesInput(tensor([[1,1,1],[-1,-1,-1.0]]))
ns_mean = tensor([0.,10.1,3.1])
ns_std = tensor([1.,1.1,0.1])
x,x.shape

(TensorSequencesInput([[ 1.,  1.,  1.],
         [-1., -1., -1.]]),
 torch.Size([2, 3]))

In [ ]:
seq_noise = SeqNoiseInjection(std=ns_std,mean=ns_mean)
seq_noise(x)

TensorSequencesInput([[ 1.,  1.,  1.],
        [-1., -1., -1.]])

In [ ]:
seq_noise = SeqNoiseInjection(std=ns_std*10)
seq_noise(x)

TensorSequencesInput([[ 1.,  1.,  1.],
        [-1., -1., -1.]])

In [ ]:
#export
class SeqNoiseInjection_Varying(RandTransform):
    split_idx=0
    '''Adds normal distributed noise to the tensor sequence with a normal distributed standard deviation for every application'''
    def __init__(self, std_std=0.1,p=1.0):
        super().__init__(p=p)
        self.std_std = tensor(std_std).type(torch.float)
        
    def encodes(self, o:TensorSequencesInput): 
        if o.device != self.std_std.device:
            self.std_std = self.std_std.to(o.device)
            
        #expand creates a view on a tensor and is therefore very fast compared to copy
        std = torch.normal(mean=0,std=self.std_std).abs()
        return o+torch.normal(mean=0,std=std.expand_as(o))

In [ ]:
x = TensorSequencesInput(tensor([[0,0,0],[0,0,0]]))
ns_std = tensor([1.,1.1,0.1])
x,x.shape

(TensorSequencesInput([[0, 0, 0],
         [0, 0, 0]]),
 torch.Size([2, 3]))

In [ ]:
seq_noise = SeqNoiseInjection_Varying(std_std=ns_std)
seq_noise(x)

TensorSequencesInput([[0, 0, 0],
        [0, 0, 0]])

In [ ]:
#export
class SeqNoiseInjection_Grouped(RandTransform):
    split_idx=0
    '''Adds normal distributed noise to the tensor sequence with a normal distributed standard deviation for every application, every group gert'''
    def __init__(self, std_std,std_idx,p=1.0):
        super().__init__(p=p)
        self.std_std = tensor(std_std).type(torch.float)
        self.std_idx = tensor(std_idx).type(torch.long)
        
    def encodes(self, o:TensorSequencesInput): 
        if o.device != self.std_std.device:
            self.std_std = self.std_std.to(o.device)
            
        #expand creates a view on a tensor and is therefore very fast compared to copy
        std = torch.normal(mean=0,std=self.std_std).abs()[self.std_idx]
        return o+torch.normal(mean=0,std=std.expand_as(o))

In [ ]:
x = TensorSequencesInput(tensor([[0,0,0],[0,0,0]]))
ns_std = tensor([1.,1.1,0.1])
x,x.shape

(TensorSequencesInput([[0, 0, 0],
         [0, 0, 0]]),
 torch.Size([2, 3]))

In [ ]:
seq_noise = SeqNoiseInjection_Grouped(std_std=[3.,0],std_idx=[0,0,1])
seq_noise(x)

TensorSequencesInput([[0, 0, 0],
        [0, 0, 0]])

### 3.2 Sequence Bias Injection Transformation

In [ ]:
#export
class SeqBiasInjection(RandTransform):
    split_idx=0
    '''Adds a normal distributed offset to the tensor sequence with seperate mean and std for every signal'''
    def __init__(self, std=1e-1,mean=0.,p=1.0):
        super().__init__(p=p)
        self.std = tensor(std).type(torch.float)
        self.mean = tensor(mean).type(torch.float)
        
    def encodes(self, o:TensorSequencesInput): 
        if o.device != self.mean.device:
            self.std = self.std.to(o.device)
            self.mean = self.mean.to(o.device)
        
        #expand creates a view on a tensor and is therefore very fast compared to copy
        mean=self.mean.repeat((o.shape[0],1,1)).expand((o.shape[0],1,o.shape[2]))
        std= self.std.repeat((o.shape[0],1,1)).expand((o.shape[0],1,o.shape[2]))
        n = torch.normal(mean=mean, std=std).expand_as(o)
        return o+n

In [ ]:
x = TensorSequencesInput(tensor([[[1,1,1],[-1,-1,-1.0]]]))
ns_mean = tensor([0.,10.1,3.1])
ns_std = tensor([1.,1.1,0.1])
seq_bias = SeqBiasInjection(std=ns_std,mean=ns_std)
seq_bias(x)[...,0]

TensorSequencesInput([[ 1., -1.]])

In [ ]:
seq_bias.mean

tensor([1.0000, 1.1000, 0.1000])

In [ ]:
seq_bias = SeqBiasInjection(std=ns_std*10)
seq_bias(x)

TensorSequencesInput([[[ 1.,  1.,  1.],
         [-1., -1., -1.]]])

### 3.3 Normalization
`Normalize` is programmed for `TensorImage` as an input tensor. It gets. At init the variable axes need to be chosen correspondingly to the shape of your tensor.

In [ ]:
#export
@Normalize
def encodes(self, x:TensorSequencesInput): 
    if x.device != self.mean.device:
        self.mean = self.mean.to(x.device)
        self.std = self.std.to(x.device)
    return (x-self.mean) / self.std

@Normalize
def decodes(self, x:TensorSequencesInput):
    if x.device != self.mean.device:
        self.mean = self.mean.to(x.device)
        self.std = self.std.to(x.device)
    return (x*self.std + self.mean)

In [ ]:
norm = Normalize.from_stats(mean=ns_mean,std=ns_std,dim=1,ndim=2,cuda=False)
x,norm(x)

(TensorSequencesInput([[[ 1.,  1.,  1.],
          [-1., -1., -1.]]]),
 TensorSequencesInput([[[  1.0000,  -8.2727, -21.0000],
          [ -1.0000, -10.0909, -41.0000]]]))

## 4. Split in Training, Validation
Splitting kann anhand von vorher bekannten Indizes, dem Dateipfad oder anderen allgemeinen Funktion durchgeführt werden.

Splitting innerhalb einer Sequenzen sollte in der Praxis nur dann geschehen wenn eine einzige Sequenz vorhanden ist. Diese kann dann vorher manuell geteilt werden.


### 4.1 Splitting mit vorgegebenem Index

In [ ]:
splitter = IndexSplitter([1,2])
test_eq(splitter(hdf_files),[[0],[1,2]])

### 4.2 Splitting mit allgemeiner Funktion
Items, bei denen die definierte Funktion `True` zurück gibt, werden den Validierungsdatensatz zugeordnet, der Rest dem Training. In diesem Fall wird nach dem Übergeordneten Ordnernamen gesucht.

In [ ]:
splitter = FuncSplitter(lambda o: Path(o).parent.name == 'valid')
test_eq(splitter(hdf_files),[[0,1],[2]])

### 4.3 Splitting anhand des Parent-Folders
Splitter, der Explizit Training und Validierungsordner den Datensätzen zuordnet

In [ ]:
#export
def _parent_idxs(items, name): return mask2idxs(Path(o).parent.name == name for o in items)

def ParentSplitter(train_name='train', valid_name='valid'):
    "Split `items` from the parent folder names (`train_name` and `valid_name`)."
    def _inner(o, **kwargs):
        return _parent_idxs(o, train_name),_parent_idxs(o, valid_name)
    return _inner

In [ ]:
splitter = ParentSplitter()
test_eq(splitter(hdf_files),[[0,1],[2]])

### 4.4 Percentage Splitter

In [ ]:
#export
def PercentageSplitter(pct=0.8):
    "Split `items` in order in relative quantity."
    def _inner(o, **kwargs):
        split_idx=int(len(o)*pct)
        return L(range(split_idx)),L(range(split_idx,len(o)))
    return _inner

In [ ]:
splitter = PercentageSplitter(0.7)
test_eq(splitter(hdf_files),[[0,1],[2]])

### 4.5 Apply To Dictionary
In Case of the Datablock API your items are a list of dictionaries. If you want to apply a Splitter to the path stored within you need a wrapper function.

In [ ]:
#export
def ApplyToDict(fn,key='path'):
    return lambda x:fn([i[key] for i in x])

### 4.6 Valid Column
Using the 'valid' column of the Dataframe that has been created by a transformation.

In [ ]:
#export
valid_clm_splitter =  FuncSplitter(lambda o:o['valid'])

In [ ]:
valid_clm_splitter(src_dicts)

((#53101) [0,1,2,3,4,5,6,7,8,9...],
 (#26550) [53101,53102,53103,53104,53105,53106,53107,53108,53109,53110...])

## 5. Dataloaders Creation
A Datasets combines all implemented components on item level.

In [ ]:
#export
def pad_sequence(batch,sorting = False):
    '''collate_fn for padding of sequences of different lengths, use in before_batch of databunch, still quite slow'''
    #takes list of tuples as input, returns list of tuples
    sorted_batch = sorted(batch, key=lambda x: x[0].shape[0], reverse=True) if sorting else batch

    pad_func = partial(torch.nn.utils.rnn.pad_sequence,batch_first=True)
    padded_tensors = [pad_func([x[tup] for x in sorted_batch]) for tup in range(len(batch[0]))]
    padded_list = [retain_types(tuple([tup[entry] for tup in padded_tensors]),batch[0]) for entry in range(len(batch))]
    #retain types is important for decoding later back to source items
#     import pdb; pdb.set_trace()
    
    return padded_list

### 5.1 Low-Level with Transforms

In [ ]:
tfms=[  [HDF2Sequence(['current','voltage']),SeqSlice(l_slc=1),toTensorSequencesInput],
        [HDF2Sequence(['voltage']),SeqSlice(r_slc=-1),toTensorSequencesOutput]]
splits = splitter([x['path'] for x in src_dicts])
dsrc = Datasets(src_dicts,tfms=tfms,splits=splits)

In [ ]:
# %%timeit
# dsrc[0]

In [ ]:
db = dsrc.dataloaders(bs=128,after_batch=[SeqNoiseInjection(std=[1.1,0.01]),Normalize(axes=[0,1])],before_batch=pad_sequence)
db.one_batch()[0].shape

torch.Size([128, 100, 2])

### 5.2 Mid-Level with Datablock API

In [ ]:
#export
class SequenceBlock(TransformBlock):
    def __init__(self, seq_extract,padding=False):
        return super().__init__(type_tfms=[seq_extract],
                                batch_tfms=[Normalize(axes=[0,1])],
                                dls_kwargs={} if not padding else {'before_batch': pad_sequence})

    @classmethod
    @delegates(HDF2Sequence, keep=True)
    def from_hdf(cls, clm_names, seq_cls=TensorSequencesInput,padding=False, **kwargs):
        return cls(HDF2Sequence(clm_names,to_cls=seq_cls,**kwargs), padding)


In [ ]:
seq = DataBlock(blocks=(SequenceBlock.from_hdf(['current','voltage'],TensorSequencesInput,padding=True,cached=False),
                        SequenceBlock.from_hdf(['voltage'],TensorSequencesOutput,cached=False)),
                get_items=tfm_src,
                splitter=ApplyToDict(ParentSplitter()))

In [ ]:
dls = seq.dataloaders(hdf_files)

## 6. Show Batches and Results

In [ ]:
#export
def plot_sequence(axs,in_sig,targ_sig,out_sig=None,**kwargs):
    n_targ = targ_sig.shape[1]
    for j,ax in  enumerate(axs[:-1]):
        ax.plot(targ_sig[:,j])
        if out_sig is not None: 
            ax.plot(out_sig[:,j])
            ax.legend(['y','ŷ'])
            if 'ref' in kwargs:
                ax.plot(kwargs['ref'][:,j]) 
        ax.label_outer()
    axs[-1].plot(in_sig)

In [ ]:
#export
def plot_seqs_single_figure(n_samples,n_targ,samples,plot_func,outs=None,**kwargs):
    rows=max(1,((n_samples-1) // 3)+1)
    cols=min(3,n_samples)
    fig = plt.figure(figsize=(9,2*cols))
    outer_grid = fig.add_gridspec(rows, cols)
#     import pdb; pdb.set_trace()
    for i in range(n_samples):
        in_sig = samples[i][0]
        targ_sig = samples[i][1]
        if outs is not None: out_sig = outs[i][0]
        inner_grid = outer_grid[i].subgridspec(n_targ+1, 1)
        axs = [fig.add_subplot(inner_grid[j]) for j in range(n_targ+1)]
        plot_func(axs,in_sig,targ_sig,out_sig=out_sig if outs is not None else None,**kwargs)
    plt.tight_layout()

In [ ]:
#export
def plot_seqs_multi_figures(n_samples,n_targ,samples,plot_func,outs=None,**kwargs):
    for i in range(n_samples):
        fig = plt.figure(figsize=(9,3))
        axs = fig.subplots(nrows=n_targ+1,sharex=True)
        in_sig = samples[i][0]
        targ_sig = samples[i][1]
        if outs is not None:  out_sig = outs[i][0]
            
        plot_func(axs,in_sig,targ_sig,out_sig=out_sig if outs is not None else None,**kwargs)
        
        plt.tight_layout()

In [ ]:
#export
@typedispatch
def show_batch(x:TensorSequences, y:TensorSequences, samples, ctxs=None, max_n=6, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = samples[0][1].shape[1]
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_sequence, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_sequence, **kwargs)
    return ctxs

In [ ]:
#export
@typedispatch
def show_results(x:TensorSequences, y:TensorSequences, samples, outs, ctxs=None, max_n=2, **kwargs):
    n_samples = min(len(samples), max_n)
    n_targ = samples[0][1].shape[1]
    if n_samples > 3:
        #if there are more then 3 samples to plot then put them in a single figure
        plot_seqs_single_figure(n_samples,n_targ,samples,plot_sequence,outs, **kwargs)
    else:
        #if there are less then 3 samples to plot then put each in its own figure
        plot_seqs_multi_figures(n_samples,n_targ,samples,plot_sequence,outs, **kwargs)
    return ctxs

In [ ]:
dls.show_batch()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_models.ipynb.
Converted 01a_IndRNN.ipynb.
Converted 02_learner.ipynb.
Converted 03_dataloaders.ipynb.
Converted 11_dualrnn.ipynb.
Converted 12_TensorQuaternions.ipynb.
Converted 13_HPOpt.ipynb.
Converted index.ipynb.
